In [1]:
import course;course.header()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Mir\\PycharmProjects\\fufezan-lab-advanced_python_2020-21_HD_fork\\notebooks\\course_title.md'

## More advanced Pandas

In [ ]:
import pandas as pd
pd.set_option("max_columns", 3000)

import numpy as np

In [ ]:
df = pd.read_csv("../data/arabica_data_cleaned.csv", index_col=0)

In [ ]:
df.head()

# Dropping object columns

In [ ]:
 df.dtypes.head()

In [ ]:
col_to_drop = []
for col, d in zip(df.columns, df.dtypes):
    if d == 'object':
        if col not in ["Country.of.Origin", "Producer", "Processing.Method"]:
            col_to_drop.append(col)
df.drop(columns=col_to_drop, inplace=True)

# renaming the dots

In [ ]:
df.rename(
    columns={
        "Country.of.Origin": "Country of Origin", 
        "Producer": "Producer", 
        "Processing.Method": "Processing Method"
    },
    inplace=True
)

In [ ]:
df['Processing Method'].value_counts()

In [ ]:
df.sample(5)

In [ ]:
df.describe()

How to describe each Country.of.Origin separately?

In [ ]:
df.groupby("Country of Origin").describe()

# Aggregation and group operations!

In [ ]:
grp = df[['Country of Origin','Body']].groupby("Country of Origin")

In [ ]:
grp.head()

Groups behave like dataframes or series!

In [ ]:
grp.idxmax()

**Note:** index is of original dataframe. Remember if not state explicitly, you will work on a view!

In [ ]:
# get the content of a group object
grp.get_group('Vietnam')

In [ ]:
grp.mean()

Groupby can also be combined


In [ ]:
df[
    [
        'Country of Origin',
        'Body',
        'Aroma'
    ]
].groupby("Country of Origin").mean()

In [ ]:
grp_cols = ['Country of Origin',"Processing Method"]
data_cols = ['Body','Aroma']
grp = df[grp_cols + data_cols].groupby(grp_cols)

In [ ]:
grp.mean()

## Interating using groupby

In [ ]:
grp_cols = ['Country of Origin',"Processing Method"]
data_cols = ['Body','Aroma']
for name, grp in df[grp_cols + data_cols].groupby(grp_cols):
    print(name)
    display(grp)
    break

How to set Body for the first entry to 10?

In [ ]:
grp_cols = ['Country of Origin',"Processing Method"]
data_cols = ['Body','Aroma']
for name, grp in df[grp_cols + data_cols].groupby(grp_cols):
    print(name)
    display(grp.iloc[0]["Body"])
    grp.iloc[0]["Body"] = 10
    break

In [ ]:
grp_cols = ['Country of Origin',"Processing Method"]
data_cols = ['Body','Aroma']
for name, grp in df[grp_cols + data_cols].groupby(grp_cols):
    print(name)
    display(grp)
    break

How to fix it?

In [ ]:
grp_cols = ['Country of Origin',"Processing Method"]
data_cols = ['Body','Aroma']
for name, grp in df[grp_cols + data_cols].groupby(grp_cols):
    print(name)
#     print()
    df.loc[grp.index[0], 'Body'] = 10
    display(grp)
    break

# more generic groupby and aggregation

In [ ]:
grps = df.groupby(['Country of Origin',"Processing Method"])
# making is simpler - not defining data columns, ie keeping all

In [ ]:
grps.agg("mean")

In [ ]:
grps.max("mean")['Body']

# Apply - Split - Combine
a very frequent usage of groupby

In [ ]:
def top_x(df, x=7, col="Body"):
    return df.sort_values(col, ascending=False).head(x)

top_x(df)

In [ ]:
df.groupby(['Country of Origin',"Processing Method"]).apply(top_x)

In [ ]:
# passing args n kwargs to apply function
df.groupby(
    ['Country of Origin',"Processing Method"]
).apply(top_x, x=2)

# Binning values
Pandas cut or pandas qcut!
* cut : cut an array by equal wide bins
* qcut : cut an array by equally populated bin

In [ ]:
pd.cut(df.Aroma, 5) 
# note dot nomenclature (instead of df['Aroma']) only works if columns
# have no spaces 

In [ ]:
pd.qcut(df.Aroma, 5)

In [ ]:
df['Aroma_bin'] = pd.qcut(df.Aroma, 5)

In [ ]:
df['Aroma_cat'] = pd.qcut(
    df.Aroma, 
    5, 
    labels=['--', '-', '', '+', '++']
)

In [ ]:
df.head()

In [ ]:
df.groupby(
    ['Country of Origin',"Processing Method", "Aroma_cat"]
).apply(top_x, x=2).head(10)

# Correlations based on Country of Origin


In [ ]:
def correlate(grp, col_1="Aroma", col_2="Aroma"):
    return grp[col_1].corr(grp[col_2])

df.groupby(
    ['Country of Origin']
).apply(
    correlate, 
    col_1="Aroma", 
    col_2="Body"
).sort_values()

In [ ]:
def correlate(grp, col_1="Aroma", col_2="Aroma"):
    return grp[col_1].corr(grp[col_2]), grp.shape[0]  # how many obervations

df.groupby(
    ['Country of Origin']
).apply(
    correlate, 
    col_1="Aroma", 
    col_2="Body"
).sort_values()

In [ ]:
def correlate(grp, col_1="Aroma", col_2="Aroma"):
    return pd.Series(
        [
            grp[col_1].corr(grp[col_2]), 
            grp.shape[0]
        ], 
        index=['pearson-corr','n']
    )

df.groupby('Country of Origin').apply(
    correlate,  
    col_1="Aroma", 
    col_2="Body"
).sort_values('pearson-corr')

# Pivot tables


In [ ]:
df.shape

In [ ]:
df.pivot_table(
    index=[
        "Country of Origin", 
        "Producer", 
        "Processing Method"
    ], 
    aggfunc=np.mean
)

In [ ]:
df.pivot_table(
    index=[
        "Processing Method",
        "Country of Origin", 
#         "Producer",  # will be dropped
    ], 
    aggfunc=np.mean
)

In [ ]:
df.pivot_table(
    index=[
        "Processing Method",
        "Country of Origin", 
#         "Producer",  # will be dropped
    ], 
    aggfunc={
        'Aroma' : np.mean,
        'Body': np.max
    }
)

In [ ]:
df.pivot_table(
    index=[
        "Processing Method",
        "Country of Origin", 
#         "Producer",  # will be dropped
    ], 
    aggfunc=[np.median, np.mean]
)

In [ ]:
idx = pd.IndexSlice

In [ ]:
_df = df.pivot_table(
    index=[
        "Processing Method",
        "Country of Origin", 
#         "Producer",  # will be dropped
    ], 
    aggfunc=[np.median, np.mean, np.min, np.max]
)
_df.loc[
    idx[:,'Brazil'], 
    idx[:,['Acidity', 'Body']]
]

In [ ]:
_df.loc[
    idx['Natural / Dry',:], 
    idx['median',['Acidity', 'Body']]
].head(5)

## Backup


In [ ]:
df.sort_values("Total.Cup.Points", ascending=False).head(1)

In [ ]:
df['Country.of.Origin'].unique()

In [ ]:
df['altitude_mean_meters'].describe()

In [ ]:
df.sort_values('altitude_mean_meters', ascending=False).head(10)

# Visualizing distribution of altitude

In [ ]:
data = []
for name, grp in df.groupby("Country.of.Origin"):
    data.append(go.Histogram(x=grp['altitude_mean_meters'], name=name))
fig = go.Figure(data=data)
fig.show()

# Reviews

In [ ]:
df.columns

In [ ]:
rcols = [
    'Aroma', 
    'Flavor', 
    'Aftertaste', 
    'Acidity', 
    'Body',
#     'Balance', 
#     'Uniformity', 
#     'Clean.Cup', 
#     'Sweetness', 
    'Cupper.Points'
]

In [ ]:
import numpy as np
data = []
for name, grp in df.groupby("Country.of.Origin"):
    s1 = grp[rcols].apply(np.max, axis=0)
    data.append(
        go.Scatterpolar(
          r=s1,
          theta=rcols,
#           fill='toself',
          name=name
        )
    )
fig = go.Figure(data=data)
fig.show()